# DEMO 1
## Draw Random Body Shapes from the Learned PCA Space

In this tutorial, we will introduce the LISST body model shape space, and how to draw random **bone lengths** from the learned PCA space.

Note that the `LISST` package should be pre-installed.

In [1]:
# load necessary modules
import os, sys, glob
import numpy as np
import torch
from lisst.utils.config_creator import ConfigLoader
from lisst.models.body import LISSTCore

DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## The LISST body configurations

To this end, there are three configurations:
- `LISST_SHAPER_v0`: has the same skeleton topology with the CMU skeleton
- `LISST_SHAPER_v1`: additionally add the nose joint to better represent the head rotation. See documents.
- `LISST_SHAPER_v2`: additionally add the nose and the heels.


In [11]:
modelcfg = 'LISST_SHAPER_v2' # specify the model configuration

shaper_config_0 = ConfigLoader('/home/yzhang/workspaces/LISST-dev/lisst/cfg/{}.yml'.format(modelcfg))
shaper = LISSTCore(shaper_config_0.modelconfig)
shaper.eval()
shaper.to(DEVICE)
## load checkpoint
shaper_ckpt_0 = '/home/yzhang/workspaces/LISST-dev/results/lisst/{}/checkpoints/epoch-000.ckp'.format(modelcfg)
shaper.load(shaper_ckpt_0)


# print info of the loaded LISST model
nj = shaper.num_kpts
jnames = shaper.joint_names
children_table = shaper.children_table

print('-- num_joints = {:d}'.format(nj))
print('-- joint_names = {}'.format(jnames))
print('-- children_table = \{ parent_joint: [children_joint]  \} ')
print(children_table)


-- successfully loaded: /home/yzhang/workspaces/LISST-dev/results/lisst/LISST_SHAPER_v2/checkpoints/epoch-000.ckp
-- num_joints = 34
-- joint_names = ['root', 'lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb', 'nose', 'lheel', 'rheel']
-- children_table = \{ parent_joint: [children_joint]  \} 
{'root': ['lhipjoint', 'rhipjoint', 'lowerback'], 'lhipjoint': ['lfemur'], 'lfemur': ['ltibia'], 'ltibia': ['lfoot', 'lheel'], 'lfoot': ['ltoes'], 'ltoes': [], 'rhipjoint': ['rfemur'], 'rfemur': ['rtibia'], 'rtibia': ['rfoot', 'rheel'], 'rfoot': ['rtoes'], 'rtoes': [], 'lowerback': ['upperback'], 'upperback': ['thorax'], 'thorax': ['lowerneck', 'lclavicle', 'rclavicle'], 'lowerneck': ['upperneck'], 'upperneck': ['head'], 'head

## sample random body shapes and visualize
Here we draw random numbers along the principle component, and then visaulize the skeleton with the body coordinate.
Note that the body is at the rest pose without global transformation. Namely, in the `canonical configuration`, the body rest pose is starish, and has the coordinate located on the pelvis.


In [12]:
# draw samaples on the 0-th (with largest eigenval) principle component, we draw 16 samples
jts = shaper.random_skeleton(pc=0, n_samples=16, device=DEVICE) # returns [N, J, 3]
jts = jts.detach().cpu().numpy()

In [14]:
# visualization
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(jts[0])
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.25)
o3d.visualization.draw_geometries([pcd, coord])
